In [ ]:
import torch
import numpy as np
from hydra import compose, initialize
from omegaconf import OmegaConf
initialize(config_path="../../configs", version_base="1.3")
cfg = compose(config_name="eval_sim")

import copy
import os

import pybullet as p
import torch
import tqdm
from rpad.partnet_mobility_utils.data import PMObject

from open_anything_diffusion.metrics.trajectory import flow_metrics
from open_anything_diffusion.models.flow_diffuser_hispndit import (
    FlowTrajectoryDiffuserSimulationModule_HisPNDiT,
)
from open_anything_diffusion.models.modules.dit_models import PN2HisDiT
from open_anything_diffusion.models.modules.history_encoder import HistoryEncoder
from open_anything_diffusion.simulations.suction import GTFlowModel, PMSuctionSim

In [ ]:
network = {
    "DiT": PN2HisDiT(
        history_embed_dim=128,
        in_channels=3,
        depth=5,
        hidden_size=128,
        num_heads=4,
        # depth=8,
        # hidden_size=256,
        # num_heads=4,
        learn_sigma=True,
    ).cuda(),
    "History": HistoryEncoder(
        history_dim=128,
        history_len=1,
        batch_norm=True,
        transformer=False,
        repeat_dim=False,
    ).cuda(),
}
ckpt_file = "/home/yishu/open_anything_diffusion/logs/train_trajectory_diffuser_hispndit/2024-05-25/02-00-54/checkpoints/epoch=399-step=331600-val_loss=0.00-weights-only.ckpt"
model = FlowTrajectoryDiffuserSimulationModule_HisPNDiT(
    network, inference_cfg=cfg.inference, model_cfg=cfg.model
).cuda()
model.load_from_ckpt(ckpt_file)
model.eval()

In [ ]:
obj_id = "8877"
joint_id = 0
pm_dir = os.path.expanduser("~/datasets/partnet-mobility/convex")

raw_data = PMObject(os.path.join(pm_dir, obj_id))
available_joints = raw_data.semantics.by_type("hinge") + raw_data.semantics.by_type(
    "slider"
)
available_joints = [joint.name for joint in available_joints]

env = PMSuctionSim(obj_id, pm_dir, gui=False)
gt_model = GTFlowModel(raw_data, env)

env.reset()

In [ ]:
for joint in available_joints:  # Close all joints
    info = p.getJointInfo(
        env.render_env.obj_id,
        env.render_env.link_name_to_index[joint],
        env.render_env.client_id,
    )
    init_angle, target_angle = info[8], info[9]
    env.set_joint_state(joint, init_angle)
    # print(init_angle, target_angle)

target_link = available_joints[joint_id]
info = p.getJointInfo(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    env.render_env.client_id,
)
init_angle, target_angle = info[8], info[9]
print(init_angle, target_angle)

In [ ]:
pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

In [ ]:
def get_local_point(object_id, link_index, world_point):
    if link_index == -1:
        # Base link (root link)
        position, orientation = p.getBasePositionAndOrientation(object_id)
    else:
        # Specific link
        link_state = p.getLinkState(object_id, link_index)
        position = link_state[4]  # Link world position
        orientation = link_state[5]  # Link world orientation

    # Convert orientation to a rotation matrix
    rotation_matrix = p.getMatrixFromQuaternion(orientation)
    rotation_matrix = np.array(rotation_matrix).reshape(3, 3)

    # Transform the world point to local coordinates
    local_point = np.dot(
        np.linalg.inv(rotation_matrix), (world_point - np.array(position))
    )
    return local_point


def get_world_point(object_id, link_index, local_point):
    if link_index == -1:
        # Base link (root link)
        position, orientation = p.getBasePositionAndOrientation(object_id)
    else:
        # Specific link
        link_state = p.getLinkState(object_id, link_index)
        position = link_state[4]  # Link world position
        orientation = link_state[5]  # Link world orientation

    # Convert orientation to a rotation matrix
    rotation_matrix = p.getMatrixFromQuaternion(orientation)
    rotation_matrix = np.array(rotation_matrix).reshape(3, 3)

    # Transform the local point to world coordinates
    world_point = np.dot(rotation_matrix, local_point) + np.array(position)
    return world_point

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([P_world]),
    torch.as_tensor([np.zeros((1200, 3))]),
    "red",
)
fig = animation.animate()
fig.show()

In [ ]:
gripper_tip_pos_before = np.array([-0.03, 0.03, 0.93])
gripper_object_contact_local = get_local_point(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    gripper_tip_pos_before,
)

In [ ]:
env.set_joint_state(target_link, -1.51019629)
pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

In [ ]:
gripper_tip_pos_after = get_world_point(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    gripper_object_contact_local,
)

In [ ]:
print(gripper_tip_pos_before, gripper_tip_pos_after)
delta_gripper = (gripper_tip_pos_before - gripper_tip_pos_after)
delta_gripper = delta_gripper / np.linalg.norm(delta_gripper) * 10

In [ ]:
import torch
import numpy as np
from flowbot3d.grasping.agents.flowbot3d import FlowNetAnimation
animation = FlowNetAnimation()
animation.add_trace(
    torch.as_tensor(P_world),
    # torch.as_tensor([pcd[mask]]),
    # torch.as_tensor([flow[mask]]),
    torch.as_tensor([gripper_tip_pos_after[None, :]]),
    torch.as_tensor([delta_gripper[None, :]]),
    "red",
)
fig = animation.animate()
fig.show()

## Visualize the simulation process to debug

In [ ]:
from open_anything_diffusion.simulations.suction import *

In [ ]:
for joint in available_joints:  # Close all joints
    info = p.getJointInfo(
        env.render_env.obj_id,
        env.render_env.link_name_to_index[joint],
        env.render_env.client_id,
    )
    init_angle, target_angle = info[8], info[9]
    env.set_joint_state(joint, init_angle)
    # print(init_angle, target_angle)

target_link = available_joints[joint_id]
info = p.getJointInfo(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    env.render_env.client_id,
)
init_angle, target_angle = info[8], info[9]
print(init_angle, target_angle)

In [ ]:
n_steps = 30
n_pts = 1200
save_name = "8877_link_1"
website = True
gui = False
gt_model = None

torch.set_printoptions(precision=10)  # Set higher precision for PyTorch outputs
np.set_printoptions(precision=10)
# p.setPhysicsEngineParameter(numSolverIterations=10)
# p.setPhysicsEngineParameter(contactBreakingThreshold=0.01, contactSlop=0.001)

initial_movement_thres = 0.01
max_trial_per_step = 50
this_step_trial = 0

sim_trajectory = [0.0] + [0] * (n_steps)  # start from 0.05
correct_direction_stack = []  # The direction stack

if website:
    # Flow animation
    animation = FlowNetAnimation()

# First, reset the environment.
env.reset()
# Joint information
info = p.getJointInfo(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    env.render_env.client_id,
)
init_angle, target_angle = info[8], info[9]

if (
    raw_data.category == "Door"
    and raw_data.semantics.by_name(target_link).type == "hinge"
):
    env.set_joint_state(target_link, init_angle + 0.0 * (target_angle - init_angle))
    # env.set_joint_state(target_link, 0.2)

if raw_data.semantics.by_name(target_link).type == "hinge":
    env.set_joint_state(target_link, init_angle + 0.0 * (target_angle - init_angle))
    # env.set_joint_state(target_link, 0.05)

# Predict the flow on the observation.
pc_obs = env.render(filter_nonobj_pts=True, n_pts=n_pts)
rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

if init_angle == target_angle:  # Not movable
    p.disconnect(physicsClientId=env.render_env.client_id)
    print("init == target")

# breakpoint()
if gt_model is None:  # GT Flow model
    pred_trajectory = model(copy.deepcopy(pc_obs))
else:
    movable_mask = gt_model.get_movable_mask(pc_obs)
    pred_trajectory = model(copy.deepcopy(pc_obs), movable_mask)
# pred_trajectory = model(copy.deepcopy(pc_obs))
# breakpoint()
pred_trajectory = pred_trajectory.reshape(
    pred_trajectory.shape[0], -1, pred_trajectory.shape[-1]
)
traj_len = pred_trajectory.shape[1]  # Trajectory length
print(f"Predicting {traj_len} length trajectories.")
pred_flow = pred_trajectory[:, 0, :]

# flow_fig(torch.from_numpy(P_world), pred_flow, sizeref=0.1, use_v2=True).show()
# breakpoint()

# Filter down just the points on the target link.
link_ixs = pc_seg == env.render_env.link_name_to_index[target_link]
# assert link_ixs.any()
if not link_ixs.any():
    p.disconnect(physicsClientId=env.render_env.client_id)
    print("link_ixs finds no point")
    animation_results = animation.animate() if website else None


if website:
    if gui:
        # Record simulation video
        log_id = p.startStateLogging(
            p.STATE_LOGGING_VIDEO_MP4,
            f"./logs/simu_eval/video_assets/{save_name}.mp4",
        )
    else:
        video_file = f"./logs/simu_eval/video_assets/{save_name}.mp4"
        # # cv2 output videos won't show on website
        frame_width = 640
        frame_height = 480
        # fps = 5
        # fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        # videoWriter = cv2.VideoWriter(video_file, fourcc, fps, (frame_width, frame_height))
        # videoWriter.write(rgbImgOpenCV)

        # Camera param
        writer = imageio.get_writer(video_file, fps=5)

        # Capture image
        width, height, rgbImg, depthImg, segImg = p.getCameraImage(
            width=frame_width,
            height=frame_height,
            viewMatrix=p.computeViewMatrixFromYawPitchRoll(
                cameraTargetPosition=[0, 0, 0],
                distance=5,
                # yaw=270,
                yaw=90,
                pitch=-30,
                roll=0,
                upAxisIndex=2,
            ),
            projectionMatrix=p.computeProjectionMatrixFOV(
                fov=60,
                aspect=float(frame_width) / frame_height,
                nearVal=0.1,
                farVal=100.0,
            ),
        )
        image = np.array(rgbImg, dtype=np.uint8)
        image = image[:, :, :3]

        # Add the frame to the video
        writer.append_data(image)

# The attachment point is the point with the highest flow.
# best_flow_ix = pred_flow[link_ixs].norm(dim=-1).argmax()
best_flow_ixs, best_flows, best_points = choose_grasp_points(
    pred_flow[link_ixs], P_world[link_ixs], filter_edge=False, k=40
)

# Teleport to an approach pose, approach, the object and grasp.
if website and not gui:
    # contact = env.teleport_and_approach(best_point, best_flow, video_writer=writer)
    best_flow_ix_id, contact = env.teleport(
        best_points, best_flows, video_writer=writer, target_link=target_link
    )
else:
    # contact = env.teleport_and_approach(best_point, best_flow)
    best_flow_ix_id, contact = env.teleport(best_points, best_flows, target_link=target_link)

best_flow = pred_flow[link_ixs][best_flow_ixs[best_flow_ix_id]]
best_point = P_world[link_ixs][best_flow_ixs[best_flow_ix_id]]

if not contact:
    if website:
        segmented_flow = np.zeros_like(pred_flow)
        segmented_flow[link_ixs] = pred_flow[link_ixs]
        segmented_flow = np.array(
            normalize_trajectory(
                torch.from_numpy(np.expand_dims(segmented_flow, 1))
            ).squeeze()
        )
        animation.add_trace(
            torch.as_tensor(P_world),
            torch.as_tensor([P_world]),
            torch.as_tensor([segmented_flow]),
            "red",
        )
        if gui:
            p.stopStateLogging(log_id)
        else:
            # Write video
            writer.close()
            # videoWriter.release()

    print("No contact!")
    p.disconnect(physicsClientId=env.render_env.client_id)
    animation_results = None if not website else animation.animate()

env.attach()
use_history = False
# gripper_tip_pos_before, _ = p.getBasePositionAndOrientation(env.gripper.base_id)
# points = p.getContactPoints(bodyA=env.gripper.body_id, bodyB=env.render_env.obj_id, linkIndexA=0)
# assert len(points)!=0, "Contact is None!!!!"
# gripper_tip_pos_before, _ = points[0][5], points[0][6]
gripper_tip_pos_before = best_point
gripper_object_contact_local = get_local_point(
    env.render_env.obj_id,
    env.render_env.link_name_to_index[target_link],
    gripper_tip_pos_before,
)
# print(gripper_tip_pos_before, gripper_object_contact_local, get_world_point(env.render_env.obj_id, env.render_env.link_name_to_index[target_link], gripper_object_contact_local))
# env.pull(best_flow)
reset = env.pull_with_constraint(best_flow, target_link=target_link)
if not reset:
    env.attach()
    gripper_tip_pos_after = get_world_point(
        env.render_env.obj_id,
        env.render_env.link_name_to_index[target_link],
        gripper_object_contact_local,
    )

    delta_gripper = np.array(gripper_tip_pos_after) - np.array(
        gripper_tip_pos_before
    )
    last_step_grasp_point = best_point
    
    if np.linalg.norm(delta_gripper) > 1e-6:  # Because
        correct_direction_stack.append(delta_gripper)
    # Judge whether the movement is
    if np.linalg.norm(delta_gripper) > initial_movement_thres:
        use_history = True
        prev_flow_pred = pred_flow.clone()  # History flow
        prev_point_cloud = copy.deepcopy(P_world)  # History point cloud

        # correct_direction_stack.append(delta_gripper)
        # print("Pushing to correct direction stack:::::", np.linalg.norm(delta_gripper), delta_gripper / np.linalg.norm(delta_gripper), best_flow / np.linalg.norm(best_flow), np.dot(delta_gripper / np.linalg.norm(delta_gripper), best_flow / np.linalg.norm(best_flow)))
else:  # Need a reset because hit the lower boundary - definitely not a good step
    use_history = False
    last_step_grasp_point = None  # No contact anymore

In [ ]:
writer.close()

In [ ]:
print("current stack top: ", np.array(correct_direction_stack[-1]) / np.linalg.norm(np.array(correct_direction_stack[-1])))
print("best flow: ", best_flow / np.linalg.norm(best_flow.numpy()))
print("cosine:", np.dot(np.array(correct_direction_stack[-1]) / np.linalg.norm(np.array(correct_direction_stack[-1])), best_flow.numpy() / np.linalg.norm(best_flow.numpy())))
success, sim_trajectory[0] = env.detect_success(target_link)
print("Current angle:", sim_trajectory[0])

In [ ]:
# breakpoint()
global_step = 1

# for i in range(n_steps):
while not success and global_step < n_steps:
    pc_obs = env.render(
        filter_nonobj_pts=True, n_pts=n_pts
    )  # Render a new point cloud!  #
    # Predict the flow on the observation.
    if gt_model is None:  # GT Flow model
        if use_history:
            print("Using history!")
            # Use history model
            pred_trajectory = model_with_history(
                copy.deepcopy(pc_obs),
                copy.deepcopy(prev_point_cloud),
                copy.deepcopy(prev_flow_pred.numpy()),
            )
        else:
            pred_trajectory = model(copy.deepcopy(pc_obs))
    else:
        movable_mask = gt_model.get_movable_mask(pc_obs)
        # breakpoint()
        pred_trajectory = model(pc_obs, movable_mask)
        # pred_trajectory = model(pc_obs)
    pred_trajectory = pred_trajectory.reshape(
        pred_trajectory.shape[0], -1, pred_trajectory.shape[-1]
    )

    pred_flow = pred_trajectory[:, 0, :]
    rgb, depth, seg, P_cam, P_world, pc_seg, segmap = pc_obs

    # Filter down just the points on the target link.
    # breakpoint()
    link_ixs = pc_seg == env.render_env.link_name_to_index[target_link]
    # assert link_ixs.any()
    if not link_ixs.any():
        if website:
            if gui:
                p.stopStateLogging(log_id)
            else:
                writer.close()
                # videoWriter.release()
        p.disconnect(physicsClientId=env.render_env.client_id)
        print("link_ixs finds no point")
        animation_results = animation.animate() if website else None
        return (
            animation_results,
            TrialResult(
                assertion=False,
                success=False,
                contact=False,
                init_angle=0,
                final_angle=0,
                now_angle=0,
                metric=0,
            ),
            sim_trajectory,
        )

    # Get the best direction.
    # best_flow_ix = pred_flow[link_ixs].norm(dim=-1).argmax()
    # ------------DEBUG-------------
    gt_model_debug = GTTrajectoryModel(raw_data, env, 1)
    gt_flow = gt_model_debug.get_gt_force_vector(pc_obs, link_ixs)
    if len(correct_direction_stack) != 0:
        print("GT flow's cosine with the last consistent vector!!!!!", np.dot(gt_flow.numpy(), correct_direction_stack[-1] / (np.linalg.norm(correct_direction_stack[-1]) + 1e-6)))
    # ------------DEBUG-------------


    best_flow_ixs, best_flows, best_points = choose_grasp_points(
        pred_flow[link_ixs],
        P_world[link_ixs],
        filter_edge=False,
        k=40,
        last_correct_direction=None
        if len(correct_direction_stack) == 0
        else correct_direction_stack[-1],
    )
    have_to_execute_incorrect = False

    if (
        len(best_flows) == 0
    ):  # All top 20 points are filtered out! - Not a good prediction - move on!
        this_step_trial += 1
        if (
            this_step_trial > max_trial_per_step
        ):  # To make the process go on, must make an action!
            have_to_execute_incorrect = True
            print("has to execute incorrect!!!")
            best_flow_ixs, best_flows, best_points = choose_grasp_points(
                pred_flow[link_ixs],
                P_world[link_ixs],
                filter_edge=False,
                k=20,
                last_correct_direction=None,
            )
        else:
            continue

    # (1) Strategy 1 - Don't change grasp point
    # (2) Strategy 2 - Change grasp point when leverage difference is large
    lev_diff_thres = 0.2
    no_movement_thres = -1

    # # Don't switch grasp point
    # lev_diff_thres = 100
    # no_movement_thres = -1
    # good_movement_thres = 1000

    if last_step_grasp_point is not None:  # Still grasping!
        gripper_tip_pos, _ = p.getBasePositionAndOrientation(env.gripper.body_id)
        pcd_dist = torch.tensor(P_world[link_ixs] - np.array(gripper_tip_pos)).norm(
            dim=-1
        )
        grasp_point_id = pcd_dist.argmin()
        lev_diff = best_flows.norm(dim=-1) - pred_flow[link_ixs][
            grasp_point_id
        ].norm(dim=-1)

    if (  # need to switch grasp point
        last_step_grasp_point is None or lev_diff[0] > lev_diff_thres
    ):
        env.reset_gripper()
        p.stepSimulation(
            env.render_env.client_id
        )  # Make sure the constraint is lifted

        if website and not gui:
            # contact = env.teleport_and_approach(best_point, best_flow, video_writer=writer)
            best_flow_ix_id, contact = env.teleport(
                best_points, best_flows, video_writer=writer
            )
        else:
            # contact = env.teleport_and_approach(best_point, best_flow)
            best_flow_ix_id, contact = env.teleport(best_points, best_flows)
        best_flow = pred_flow[link_ixs][best_flow_ixs[best_flow_ix_id]]
        best_point = P_world[link_ixs][best_flow_ixs[best_flow_ix_id]]
        last_step_grasp_point = best_point  # Grasp a new point
        # print("new!", last_step_grasp_point)

        if not contact:
            if website:
                segmented_flow = np.zeros_like(pred_flow)
                segmented_flow[link_ixs] = pred_flow[link_ixs]
                segmented_flow = np.array(
                    normalize_trajectory(
                        torch.from_numpy(np.expand_dims(segmented_flow, 1))
                    ).squeeze()
                )
                animation.add_trace(
                    torch.as_tensor(P_world),
                    torch.as_tensor([P_world]),
                    torch.as_tensor([segmented_flow]),
                    "red",
                )
                if gui:
                    p.stopStateLogging(log_id)
                else:
                    # Write video
                    writer.close()
                    # videoWriter.release()

            print("No contact!")
            p.disconnect(physicsClientId=env.render_env.client_id)
            animation_results = None if not website else animation.animate()
            return (
                animation_results,
                TrialResult(
                    success=False,
                    assertion=True,
                    contact=False,
                    init_angle=0,
                    final_angle=0,
                    now_angle=0,
                    metric=0,
                ),
                sim_trajectory,
            )

        env.attach()
    else:  # Stick to the old grasp point
        best_flow = pred_flow[link_ixs][best_flow_ixs[0]]
        best_point = P_world[link_ixs][grasp_point_id]
        last_step_grasp_point = (
            best_point  # The original point - don't need to change
        )
        # print("same:", last_step_grasp_point)

    # Execute the step:
    print("GT flow's cosine with the predicted vector!!!!!", gt_flow, best_flow / (np.linalg.norm(best_flow) + 1e-6), np.dot(gt_flow.numpy(), best_flow / (np.linalg.norm(best_flow) + 1e-6)))
    env.attach()
    # gripper_tip_pos_before, _ = p.getBasePositionAndOrientation(env.gripper.base_id)
    gripper_tip_pos_before = last_step_grasp_point
    gripper_object_contact_local = get_local_point(
        env.render_env.obj_id,
        env.render_env.link_name_to_index[target_link],
        gripper_tip_pos_before,
    )
    reset = env.pull_with_constraint(best_flow, target_link=target_link)
    # -------DEBUG-------
    # print(gt_flow)
    # reset = env.pull_with_constraint(gt_flow, target_link=target_link)
    # -------DEBUG-------
    if not reset:
        env.attach()
        gripper_tip_pos_after = get_world_point(
            env.render_env.obj_id,
            env.render_env.link_name_to_index[target_link],
            gripper_object_contact_local,
        )

        # Now with filter: we guarantee that every step is correct!!
        delta_gripper = np.array(gripper_tip_pos_after) - np.array(
            gripper_tip_pos_before
        )
        # print(delta_gripper, np.linalg.norm(delta_gripper))
        if (
            len(correct_direction_stack) == 0
            and np.linalg.norm(delta_gripper) < initial_movement_thres
        ):  # Still waiting for the initial movement!!!
            use_history = False
        else:
            last_step_grasp_point = best_point
            if len(correct_direction_stack) == 0 or (
                np.dot(delta_gripper, correct_direction_stack[-1]) > 0
                and np.linalg.norm(delta_gripper) > initial_movement_thres
            ):  # The actual move direction is consistent with the previous movements
                use_history = True
                if have_to_execute_incorrect and (
                    np.dot(delta_gripper, correct_direction_stack[-1]) > 0
                ):
                    print("Doesn't satisfy cosine condition, but is correct!")
                prev_flow_pred = pred_flow.clone()  # History flow
                prev_point_cloud = copy.deepcopy(P_world)  # History point cloud

                correct_direction_stack.append(
                    delta_gripper / (np.linalg.norm(delta_gripper) + 1e-6)
                )
    else:  # Reset
        use_history = False
        last_step_grasp_point = None

    global_step += 1
    this_step_trial = 0

    if website:
        # Add pcd to flow animation
        segmented_flow = np.zeros_like(pred_flow)
        segmented_flow[link_ixs] = pred_flow[link_ixs]
        segmented_flow = np.array(
            normalize_trajectory(
                torch.from_numpy(np.expand_dims(segmented_flow, 1))
            ).squeeze()
        )
        animation.add_trace(
            torch.as_tensor(P_world),
            torch.as_tensor([P_world]),
            torch.as_tensor([segmented_flow]),
            "red",
        )

        # Capture frame
        width, height, rgbImg, depthImg, segImg = p.getCameraImage(
            width=frame_width,
            height=frame_height,
            viewMatrix=p.computeViewMatrixFromYawPitchRoll(
                cameraTargetPosition=[0, 0, 0],
                distance=5,
                # yaw=270,
                yaw=90,
                pitch=-30,
                roll=0,
                upAxisIndex=2,
            ),
            projectionMatrix=p.computeProjectionMatrixFOV(
                fov=60,
                aspect=float(frame_width) / frame_height,
                nearVal=0.1,
                farVal=100.0,
            ),
        )
        # rgbImgOpenCV = cv2.cvtColor(np.array(rgbImg), cv2.COLOR_RGB2BGR)
        # videoWriter.write(rgbImgOpenCV)
        image = np.array(rgbImg, dtype=np.uint8)
        image = image[:, :, :3]

        # Add the frame to the video
        writer.append_data(image)

    success, sim_trajectory[global_step] = env.detect_success(target_link)

    if success:
        break

    # pc_obs = env.render(filter_nonobj_pts=True, n_pts=1200)   # Render a new point cloud!
    # if len(correct_direction_stack) == 2:
    #     breakpoint()

for left_step in range(global_step, 31):
    sim_trajectory[left_step] = sim_trajectory[global_step]
# calculate the metrics
# if success:
#     metric = 1
# else:
#     curr_pos = env.get_joint_value(target_link)
#     metric = (curr_pos - init_angle) / (target_angle - init_angle)
#     metric = min(max(metric, 0), 1)
curr_pos = env.get_joint_value(target_link)
metric = (curr_pos - init_angle) / (target_angle - init_angle)
metric = min(max(metric, 0), 1)

if website:
    if gui:
        p.stopStateLogging(log_id)
    else:
        writer.close()
        # videoWriter.release()

p.disconnect(physicsClientId=env.render_env.client_id)
animation_results = None if not website else animation.animate()